# In this notebook we would build the CNN model

In [ ]:
keep 
atr_14  , obv

In [ ]:
keep 
atr_14  , obv

drop

ema_cross_up, macd_cross_up, oversold_reversal, overbought_reversal, trending_market

In [ ]:
drop_cnn = [
    'open', 'high', 'low', 'typical_price', 'EMA_7', 'EMA_21', 'SMA_20', 
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower', 
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'bollinger_width', 'volatility_regime',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6''ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'
]

In [3]:
# cnn_optuna_f0_56.py  (patched)
# =============================================================
# Walk-forward Optuna search for a 1-D CNN optimised on F-β (β=0.56)
# =============================================================
import os, json, warnings, joblib, optuna, time
from pathlib import Path
from datetime import datetime
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score]
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# ─────────────────────────  runtime & seeds  ───────────────────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ────────────────────────────  config  ─────────────────────────────────
CSV_PATH   = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                  r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
VAL_RATIO  = 0.20
WF_FOLDS   = 2
BETA       = 0.56                     # precision-weighted Fβ
N_TRIALS   = 30
TIMEOUT    = 35 * 60                  # seconds
SCALER_OUT = "cnn_scaler.pkl"
PARAMS_OUT = "cnn_best_params.json"

DROP_COLS = [...]   # same long list as before (omitted for brevity)

# ─────────────────────────── helpers  ──────────────────────────────────
def f_beta(y_true, y_pred, beta=BETA):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    return 0.0 if p + r == 0 else (1 + beta**2) * p * r / (beta**2 * p + r)

def make_windows(arr, labels, win):
    X, y = [], []
    for i in range(win, len(arr)):
        X.append(arr[i - win:i])
        y.append(labels[i])
    return np.asarray(X, np.float32), np.asarray(y, np.int8)

def load_and_scale():
    df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
    df = df.loc["2018-01-01":]                       # discard early data
    df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
    df = df.dropna(subset=["target"]).dropna()

    X_all = df.drop(columns="target").values
    y_all = df["target"].astype(int).values
    split_real = int(len(df) * (1 - VAL_RATIO))

    scaler = StandardScaler().fit(X_all[:split_real])
    X_scaled = scaler.transform(X_all).astype(np.float32)
    joblib.dump(scaler, SCALER_OUT)

    print(f"✔ Scaler fitted on {split_real:,} rows | features = {X_all.shape[1]}")
    return X_scaled, y_all, split_real, X_all.shape[1]

X_SCALED, Y_ALL, REAL_SPLIT_IDX, N_FEATS = load_and_scale()

# ─────────────────────── model factory  ────────────────────────────────
def build_model(trial, win):
    conv_blocks  = trial.suggest_int("conv_blocks", 1, 3)
    base_filters = trial.suggest_int("filters", 32, 96, step=32)
    kernel       = trial.suggest_int("kernel", 2, 4)
    activation   = trial.suggest_categorical("act", ["relu", "elu", "selu"])
    dropout      = trial.suggest_float("dropout", 0.05, 0.3)
    dense_units  = trial.suggest_int("dense", 64, 128, step=64)
    l2reg        = trial.suggest_float("l2", 1e-6, 1e-3, log=True)
    pool_choice  = trial.suggest_categorical("pool", ["gap", "gmp"])
    extra_dense  = trial.suggest_categorical("extra_dense", [True, False])

    inp = layers.Input(shape=(win, N_FEATS))
    x = inp
    for b in range(conv_blocks):
        f = base_filters * (2**b)
        y = layers.Conv1D(f, kernel, padding="causal",
                          activation=activation,
                          kernel_regularizer=regularizers.l2(l2reg))(x)
        y = layers.BatchNormalization()(y)
        if x.shape[-1] != y.shape[-1]:
            x = layers.Conv1D(f, 1, padding="same")(x)
        x = layers.add([x, y])

    x = (layers.GlobalAveragePooling1D() if pool_choice == "gap"
         else layers.GlobalMaxPooling1D())(x)
    x = layers.Dropout(dropout)(x)                    # always apply dropout

    if extra_dense:
        x = layers.Dense(dense_units, activation=activation,
                         kernel_regularizer=regularizers.l2(l2reg))(x)
        x = layers.Dropout(dropout)(x)

    out = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inp, out)

# ───────────────────────  objective  ───────────────────────────────────
def objective(trial):
    win    = trial.suggest_int("window", 12, 48, step=6)
    batch  = trial.suggest_categorical("batch", [32, 64])
    lr     = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    epochs = trial.suggest_int("epochs", 20, 60, step=20)

    data_len = REAL_SPLIT_IDX - win
    val_size = data_len // (WF_FOLDS + 1)
    if val_size < 25:
        return 1.0

    f_scores, p_scores, r_scores = [], [], []

    for fold in range(WF_FOLDS):
        val_start = data_len - (WF_FOLDS - fold) * val_size
        val_end   = val_start + val_size
        if val_start < win * 2:
            continue

        X_fold = X_SCALED[:val_end + win]
        y_fold = Y_ALL[:val_end + win]
        X_win, y_win = make_windows(X_fold, y_fold, win)

        X_tr, y_tr = X_win[:val_start],           y_win[:val_start]
        X_va, y_va = X_win[val_start:val_end],    y_win[val_start:val_end]
        if len(X_tr) < 50 or len(X_va) < 25:
            continue

        tf.keras.backend.clear_session()
        model = build_model(trial, win)
        model.compile(keras.optimizers.Adam(lr), loss="binary_crossentropy")

        es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=0)
        model.fit(X_tr, y_tr,
                  epochs=epochs,
                  batch_size=batch,
                  validation_data=(X_va, y_va),
                  callbacks=[es],
                  verbose=0)

        prob = model.predict(X_va, verbose=0).ravel()
        preds = (prob >= 0.5).astype(int)

        p = precision_score(y_va, preds, zero_division=0)
        r = recall_score(y_va, preds, zero_division=0)
        f = f_beta(y_va, preds)

        p_scores.append(p); r_scores.append(r); f_scores.append(f)
        print(f"  ↪ Fold {fold}:  P={p:.3f}  R={r:.3f}  Fβ={f:.3f}")

        trial.report(np.mean(f_scores), step=fold)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    if not f_scores:
        return 1.0

    avg_p, avg_r, avg_f = map(np.mean, (p_scores, r_scores, f_scores))
    # ----- per-trial summary print -----
    print(f"✅ Trial {trial.number} done  |  "
          f"P={avg_p:.3f}  R={avg_r:.3f}  Fβ={avg_f:.3f}")
    return 1.0 - avg_f

# ────────────────────────────── run  ───────────────────────────────────
if __name__ == "__main__":
    t0 = time.time()
    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=SEED),
                                pruner=optuna.pruners.MedianPruner(n_startup_trials=5))

    study.optimize(objective, n_trials=N_TRIALS,
                   timeout=TIMEOUT, show_progress_bar=True)

    best_f = 1.0 - study.best_value
    print(f"\n🏆 Best F-β (β={BETA:.2f}): {best_f:.4f}")
    print(json.dumps(study.best_params, indent=2))

    with open(PARAMS_OUT, "w") as f:
        json.dump(study.best_params, f, indent=2)
    print(f"💾 Params → {PARAMS_OUT}   |   Scaler → {SCALER_OUT}")
    print(f"🕒 Total time: {(time.time()-t0)/60:.1f} min   "
          f"({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")


[I 2025-06-12 18:17:51,210] A new study created in memory with name: no-name-2c311586-1232-4232-a74f-0a47b3d54715


✔ Scaler fitted on 12,684 rows | features = 65


  0%|          | 0/30 [00:00<?, ?it/s]

  ↪ Fold 0:  P=0.520  R=0.774  Fβ=0.564


Best trial: 0. Best value: 0.469437:   3%|▎         | 1/30 [00:17<08:26, 17.45s/it, 17.45/2100 seconds]

...

  ↪ Fold 0:  P=0.516  R=0.477  Fβ=0.506
[I 2025-06-12 18:27:36,750] Trial 27 pruned. 


Best trial: 0. Best value: 0.469437:  97%|█████████▋| 29/30 [09:54<00:16, 16.78s/it, 594.26/2100 seconds]

  ↪ Fold 0:  P=0.501  R=0.463  Fβ=0.492
[I 2025-06-12 18:27:45,469] Trial 28 pruned. 


Best trial: 0. Best value: 0.469437: 100%|██████████| 30/30 [10:00<00:00, 20.03s/it, 600.87/2100 seconds]

  ↪ Fold 0:  P=0.512  R=0.698  Fβ=0.547
[I 2025-06-12 18:27:52,076] Trial 29 pruned. 

🏆 Best F-β (β=0.56): 0.5306
{
  "window": 24,
  "batch": 32,
  "lr": 0.0010401663679887319,
  "epochs": 20,
  "conv_blocks": 1,
  "filters": 32,
  "kernel": 4,
  "act": "elu",
  "dropout": 0.2924774630404986,
  "dense": 128,
  "l2": 4.335281794951567e-06,
  "pool": "gmp",
  "extra_dense": false
}
💾 Params → cnn_best_params.json   |   Scaler → cnn_scaler.pkl
🕒 Total time: 10.0 min   (2025-06-12 18:27:52)


In [ ]:
🏆 Top 5 Trials by F1-Score: - old param set
Rank	Trial	Precision	Recall	F1 Score	Notable Hyperparameters Summary
1️⃣	#3	0.527	0.676	0.593	window=36, batch=32, lr=0.00021, filters=64, kernel=2, act=relu, dropout=0.186, dense=64, conv_blocks=1, pool=gmp, extra_dense=True
2️⃣	#2	0.516	0.773	0.620	window=24, batch=64, lr=0.00056, filters=32, kernel=4, act=elu, dropout=0.180, dense=128, conv_blocks=2, pool=gap, extra_dense=True
3️⃣	#27	0.487	0.615	0.544	window=30, batch=32, lr=0.00202, filters=64, kernel=3, act=elu, dropout=0.147, dense=128, conv_blocks=2, pool=gmp, extra_dense=True
4️⃣	#4	0.527	0.528	0.527	window=12, batch=32, lr=0.00173, filters=64, kernel=2, act=relu, dropout=0.066, dense=64, conv_blocks=1, pool=gap, extra_dense=True
5️⃣	#24	0.528	0.399	0.454	window=18, batch=32, lr=0.00055, filters=32, kernel=4, act=elu, dropout=0.094, dense=128, conv_blocks=1, pool=gmp, extra_dense=False

In [11]:
"""
cnn_compare_param_sets.py
─────────────────────────
Trains - and compares - several 1-D CNN configurations on the 4-hour BTC data.
Each configuration is built exactly from the hyper-parameter dictionary you
supply in `PARAM_SETS`.
The script prints Precision, Recall, F1 for every run and a final leaderboard.
"""

# ═══════════════════════════════════════════════════════════════════════
# Imports & global set-up
# ═══════════════════════════════════════════════════════════════════════
import os, json, joblib, warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             accuracy_score, roc_auc_score)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ═══════════════════════════════════════════════════════════════════════
# Data paths & constants
# ═══════════════════════════════════════════════════════════════════════
CSV_PATH = Path(r"C:/Users/ADMIN/Desktop/Coding_projects/stock_market_prediction/"
                r"Stock-Market-Prediction/data/processed/"
                r"gemini_btc_with_features_4h.csv")

VAL_FRAC   = 0.20          # last 20 % is the validation window
EPOCHS     = 80            # same for every run
EARLY_STOP = 12
L2_REG_DEF = 1e-6          # fallback if a param dict misses 'l2'
CONV_BLOCKS = 1            # fixed depth – keep the test fast

# Columns to drop (same list you tuned on – prevents data leakage)
DROP_COLS = [  # … (list exactly as before – shortened for brevity here)
    'open','high','low','typical_price','EMA_7','EMA_21','SMA_20','SMA_50',
    'vwap_24h','close_4h','bollinger_upper','bollinger_lower','resistance_level',
    'support_level','high_low','high_close','low_close','true_range',
    #  … snip …
    'overbought_reversal','close'
]

# ═══════════════════════════════════════════════════════════════════════
# The six candidate hyper-parameter sets
# ═══════════════════════════════════════════════════════════════════════
PARAM_SETS = [
    {'window': 12, 'batch': 64, 'lr': 0.000898, 'filters': 64, 'kernel': 3,
     'act': 'elu',  'dropout': 0.209, 'dense': 64,  'l2': 3.35e-05,
     'pool': 'gap', 'extra_dense': False},

    {'window': 30, 'batch': 32, 'lr': 0.001545, 'filters': 96, 'kernel': 4,
     'act': 'elu',  'dropout': 0.14,  'dense': 128, 'l2': 0.000168,
     'pool': 'gmp', 'extra_dense': False},

    {'window': 36, 'batch': 32, 'lr': 0.000215, 'filters': 64, 'kernel': 2,
     'act': 'relu', 'dropout': 0.186, 'dense': 64,  'l2': 0.000255,
     'pool': 'gmp', 'extra_dense': True},

    {'window': 24, 'batch': 32, 'lr': 0.00104,  'filters': 32, 'kernel': 4,
     'act': 'elu',  'dropout': 0.292, 'dense': 128, 'l2': 4.33e-06,
     'pool': 'gmp', 'extra_dense': False},

    {'window': 30, 'batch': 64, 'lr': 0.000172, 'filters': 64, 'kernel': 4,
     'act': 'selu','dropout': 0.061, 'dense': 128, 'l2': 3.24e-06,
     'pool': 'gmp', 'extra_dense': True},

    # Duplicate of the first – still included for completeness
    {'window': 12, 'batch': 64, 'lr': 0.000898, 'filters': 64, 'kernel': 3,
     'act': 'elu',  'dropout': 0.209, 'dense': 64,  'l2': 3.35e-05,
     'pool': 'gap', 'extra_dense': False}
]

# ═══════════════════════════════════════════════════════════════════════
# Data prep helpers
# ═══════════════════════════════════════════════════════════════════════
def make_windows(arr, labels, win):
    xs, ys = [], []
    for i in range(win, len(arr)):
        xs.append(arr[i-win:i])
        ys.append(labels[i])
    return (np.asarray(xs, np.float32), np.asarray(ys, np.int8))

# ═══════════════════════════════════════════════════════════════════════
# Model factory
# ═══════════════════════════════════════════════════════════════════════
def build_model(cfg, n_features):
    """Construct a simple residual-style 1-D CNN from cfg dict."""
    l2 = cfg.get("l2", L2_REG_DEF)
    inp = layers.Input(shape=(cfg["window"], n_features))
    x   = inp

    for b in range(CONV_BLOCKS):
        f = cfg["filters"] * (2 ** b)
        y = layers.Conv1D(f, cfg["kernel"], padding="causal",
                          activation=cfg["act"],
                          kernel_regularizer=regularizers.l2(l2))(x)
        y = layers.BatchNormalization()(y)

        if CONV_BLOCKS > 1:
            if x.shape[-1] != y.shape[-1]:
                x = layers.Conv1D(f, 1, padding="same")(x)
            x = layers.Add()([x, y])
        else:
            x = y

    pool = layers.GlobalMaxPooling1D if cfg["pool"] == "gmp" else layers.GlobalAveragePooling1D
    x    = pool()(x)
    x    = layers.Dropout(cfg["dropout"])(x)

    if cfg["extra_dense"]:
        x = layers.Dense(cfg["dense"], activation=cfg["act"],
                         kernel_regularizer=regularizers.l2(l2))(x)
        x = layers.Dropout(cfg["dropout"])(x)

    out = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inp, out)

# ═══════════════════════════════════════════════════════════════════════
# 1. Load / scale data once
# ═══════════════════════════════════════════════════════════════════════
print("📊 Loading & scaling data …")
df = (pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
        .loc["2018-01-01":]
        .drop(columns=[c for c in DROP_COLS if c in df.columns])
        .dropna(subset=["target"]).dropna())

X_raw = df.drop(columns="target").values
y_raw = df["target"].astype(int).values
n_features = X_raw.shape[1]

split = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split])
X_train_scaled = scaler.transform(X_raw[:split]).astype(np.float32)
X_val_scaled   = scaler.transform(X_raw[split:]).astype(np.float32)
y_train_raw    = y_raw[:split]
y_val_raw      = y_raw[split:]

# ═══════════════════════════════════════════════════════════════════════
# 2. Iterate over parameter sets
# ═══════════════════════════════════════════════════════════════════════
results = []

for idx, cfg in enumerate(PARAM_SETS, 1):
    tag = f"Run-{idx:02d}  (win={cfg['window']}, filt={cfg['filters']}, pool={cfg['pool']})"
    print(f"\n{tag}\n" + "─" * len(tag))

    # --- prepare windows for this window size
    X_tr, y_tr = make_windows(X_train_scaled, y_train_raw, cfg["window"])
    X_va, y_va = make_windows(X_val_scaled,   y_val_raw,   cfg["window"])

    # --- build & train
    tf.keras.backend.clear_session()
    model = build_model(cfg, n_features)
    model.compile(optimizer=keras.optimizers.Adam(cfg["lr"]),
                  loss="binary_crossentropy")

    cb = [keras.callbacks.EarlyStopping(patience=EARLY_STOP,
                                        restore_best_weights=True,
                                        verbose=0)]
    model.fit(X_tr, y_tr,
              epochs=EPOCHS,
              batch_size=cfg["batch"],
              validation_data=(X_va, y_va),
              callbacks=cb,
              verbose=0)

    # --- evaluate
    prob = model.predict(X_va, verbose=0).ravel()
    pred = (prob >= 0.5).astype(int)

    prec = precision_score(y_va, pred, zero_division=0)
    rec  = recall_score(y_va, pred,    zero_division=0)
    f1   = f1_score(y_va, pred,        zero_division=0)

    print(f"Precision : {prec:5.3f}   Recall : {rec:5.3f}   F1 : {f1:5.3f}")

    results.append({
        **cfg,
        "precision": prec,
        "recall"   : rec,
        "f1"       : f1,
        "auc"      : roc_auc_score(y_va, prob)
    })

# ═══════════════════════════════════════════════════════════════════════
# 3. Leaderboard
# ═══════════════════════════════════════════════════════════════════════
print("\n🏆  Leaderboard (sorted by F1)")
results_sorted = sorted(results, key=lambda d: d["f1"], reverse=True)

for rk, res in enumerate(results_sorted, 1):
    print(f"{rk:>2}. F1={res['f1']:.3f}  "
          f"P={res['precision']:.3f}  R={res['recall']:.3f}  "
          f"(win={res['window']}, filt={res['filters']}, pool={res['pool']})")

# ═══════════════════════════════════════════════════════════════════════
# 4. Optional – save summary JSON
# ═══════════════════════════════════════════════════════════════════════
summary_path = "cnn_param_comparison_summary.json"
with open(summary_path, "w") as fp:
    json.dump({
        "timestamp" : datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "metrics"   : results_sorted
    }, fp, indent=2)

print(f"\n📑 Comparison summary saved → {summary_path}")


📊 Loading & scaling data …

Run-01  (win=12, filt=64, pool=gap)
───────────────────────────────────
Precision : 0.530   Recall : 0.492   F1 : 0.511

Run-02  (win=30, filt=96, pool=gmp)
───────────────────────────────────
Precision : 0.491   Recall : 0.144   F1 : 0.223

Run-03  (win=36, filt=64, pool=gmp)
───────────────────────────────────
Precision : 0.515   Recall : 0.535   F1 : 0.525

Run-04  (win=24, filt=32, pool=gmp)
───────────────────────────────────
Precision : 0.525   Recall : 0.659   F1 : 0.584

Run-05  (win=30, filt=64, pool=gmp)
───────────────────────────────────
Precision : 0.535   Recall : 0.138   F1 : 0.220

Run-06  (win=12, filt=64, pool=gap)
───────────────────────────────────
Precision : 0.530   Recall : 0.437   F1 : 0.479

🏆  Leaderboard (sorted by F1)
 1. F1=0.584  P=0.525  R=0.659  (win=24, filt=32, pool=gmp)
 2. F1=0.525  P=0.515  R=0.535  (win=36, filt=64, pool=gmp)
 3. F1=0.511  P=0.530  R=0.492  (win=12, filt=64, pool=gap)
 4. F1=0.479  P=0.530  R=0.437  (win

In [5]:
"""
cnn_compare_param_sets.py
─────────────────────────
Trains - and compares - several 1-D CNN configurations on the 4-hour BTC data.
Each configuration is built exactly from the hyper-parameter dictionary you
supply in `PARAM_SETS`.
The script prints Precision, Recall, F1 for every run and a final leaderboard.
"""

# ═══════════════════════════════════════════════════════════════════════
# Imports & global set-up
# ═══════════════════════════════════════════════════════════════════════
import os, json, joblib, warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             accuracy_score, roc_auc_score)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ═══════════════════════════════════════════════════════════════════════
# Data paths & constants
# ═══════════════════════════════════════════════════════════════════════
CSV_PATH = Path(r"C:/Users/ADMIN/Desktop/Coding_projects/stock_market_prediction/"
                r"Stock-Market-Prediction/data/processed/"
                r"gemini_btc_with_features_4h.csv")

VAL_FRAC   = 0.20          # last 20 % is the validation window
EPOCHS     = 80            # same for every run
EARLY_STOP = 12
L2_REG_DEF = 1e-6          # fallback if a param dict misses 'l2'
CONV_BLOCKS = 1            # fixed depth – keep the test fast

# Columns to drop (same list you tuned on – prevents data leakage)
DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_7', 'EMA_21', 'SMA_20', 
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower', 
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'bollinger_width', 'volatility_regime',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6''ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'
]
# ═══════════════════════════════════════════════════════════════════════
# The six candidate hyper-parameter sets
# ═══════════════════════════════════════════════════════════════════════
PARAM_SETS = [
    # Original sets
    {'window': 12, 'batch': 64, 'lr': 0.000898, 'filters': 64, 'kernel': 3,
     'act': 'elu',  'dropout': 0.209, 'dense': 64,  'l2': 3.35e-05,
     'pool': 'gap', 'extra_dense': False, 'conv_blocks': 1, 'epochs': 20},

    {'window': 30, 'batch': 32, 'lr': 0.001545, 'filters': 96, 'kernel': 4,
     'act': 'elu',  'dropout': 0.14,  'dense': 128, 'l2': 0.000168,
     'pool': 'gmp', 'extra_dense': False, 'conv_blocks': 1, 'epochs': 20},

    {'window': 36, 'batch': 32, 'lr': 0.000215, 'filters': 64, 'kernel': 2,
     'act': 'relu', 'dropout': 0.186, 'dense': 64,  'l2': 0.000255,
     'pool': 'gmp', 'extra_dense': True, 'conv_blocks': 1, 'epochs': 20},

    {'window': 24, 'batch': 32, 'lr': 0.00104,  'filters': 32, 'kernel': 4,
     'act': 'elu',  'dropout': 0.292, 'dense': 128, 'l2': 4.33e-06,
     'pool': 'gmp', 'extra_dense': False, 'conv_blocks': 1, 'epochs': 20},

    {'window': 30, 'batch': 64, 'lr': 0.000172, 'filters': 64, 'kernel': 4,
     'act': 'selu','dropout': 0.061, 'dense': 128, 'l2': 3.24e-06,
     'pool': 'gmp', 'extra_dense': True, 'conv_blocks': 2, 'epochs': 20},

    # NEW ADDITIONS based on optimization results
    
    # 1. Current optimization winner - highest weighted F1 (0.531)
    {'window': 24, 'batch': 32, 'lr': 0.0010401663679887319, 'filters': 32, 'kernel': 4,
     'act': 'elu', 'dropout': 0.2924774630404986, 'dense': 128, 'l2': 4.335281794951567e-06,
     'pool': 'gmp', 'extra_dense': False, 'conv_blocks': 1, 'epochs': 20},

    # 2. Trial 8 from current optimization - your training script config
    {'window': 18, 'batch': 64, 'lr': 0.0007145565133513971, 'filters': 96, 'kernel': 2,
     'act': 'selu', 'dropout': 0.1105138178778751, 'dense': 128, 'l2': 0.00019268985325226193,
     'pool': 'gmp', 'extra_dense': False, 'conv_blocks': 1, 'epochs': 20},

    # 3. Old Trial #3 - top old performer (0.593 standard F1)
    {'window': 36, 'batch': 32, 'lr': 0.000215, 'filters': 64, 'kernel': 2,
     'act': 'relu', 'dropout': 0.186, 'dense': 64, 'l2': 0.000255,
     'pool': 'gmp', 'extra_dense': True, 'conv_blocks': 1, 'epochs': 20},

    # 4. Old Trial #2 - high recall config (0.620 standard F1, 0.773 recall)
    {'window': 24, 'batch': 64, 'lr': 0.00056, 'filters': 32, 'kernel': 4,
     'act': 'elu', 'dropout': 0.180, 'dense': 128, 'l2': 1e-5,
     'pool': 'gap', 'extra_dense': True, 'conv_blocks': 2, 'epochs': 20},

    # 5. Current Trial 1 - multi-block architecture
    {'window': 30, 'batch': 64, 'lr': 0.00017258215396625024, 'filters': 64, 'kernel': 4,
     'act': 'selu', 'dropout': 0.061612603179999434, 'dense': 128, 'l2': 3.247673570627449e-06,
     'pool': 'gmp', 'extra_dense': True, 'conv_blocks': 2, 'epochs': 20},

    # 6. Old Trial #27 - balanced config
    {'window': 30, 'batch': 32, 'lr': 0.00202, 'filters': 64, 'kernel': 3,
     'act': 'elu', 'dropout': 0.147, 'dense': 128, 'l2': 5e-5,
     'pool': 'gmp', 'extra_dense': True, 'conv_blocks': 2, 'epochs': 20}
]

# ═══════════════════════════════════════════════════════════════════════
# Data prep helpers
# ═══════════════════════════════════════════════════════════════════════
def make_windows(arr, labels, win):
    xs, ys = [], []
    for i in range(win, len(arr)):
        xs.append(arr[i-win:i])
        ys.append(labels[i])
    return (np.asarray(xs, np.float32), np.asarray(ys, np.int8))

# ═══════════════════════════════════════════════════════════════════════
# Model factory
# ═══════════════════════════════════════════════════════════════════════
def build_model(cfg, n_features):
    """Construct a simple residual-style 1-D CNN from cfg dict."""
    l2 = cfg.get("l2", L2_REG_DEF)
    inp = layers.Input(shape=(cfg["window"], n_features))
    x   = inp

    for b in range(CONV_BLOCKS):
        f = cfg["filters"] * (2 ** b)
        y = layers.Conv1D(f, cfg["kernel"], padding="causal",
                          activation=cfg["act"],
                          kernel_regularizer=regularizers.l2(l2))(x)
        y = layers.BatchNormalization()(y)

        if CONV_BLOCKS > 1:
            if x.shape[-1] != y.shape[-1]:
                x = layers.Conv1D(f, 1, padding="same")(x)
            x = layers.Add()([x, y])
        else:
            x = y

    pool = layers.GlobalMaxPooling1D if cfg["pool"] == "gmp" else layers.GlobalAveragePooling1D
    x    = pool()(x)
    x    = layers.Dropout(cfg["dropout"])(x)

    if cfg["extra_dense"]:
        x = layers.Dense(cfg["dense"], activation=cfg["act"],
                         kernel_regularizer=regularizers.l2(l2))(x)
        x = layers.Dropout(cfg["dropout"])(x)

    out = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inp, out)

# ═══════════════════════════════════════════════════════════════════════
# 1. Load / scale data once
# ═══════════════════════════════════════════════════════════════════════
print("📊 Loading & scaling data …")

df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]

# Now it's safe to reference df.columns
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()


X_raw = df.drop(columns="target").values
y_raw = df["target"].astype(int).values
n_features = X_raw.shape[1]

split = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split])
X_train_scaled = scaler.transform(X_raw[:split]).astype(np.float32)
X_val_scaled   = scaler.transform(X_raw[split:]).astype(np.float32)
y_train_raw    = y_raw[:split]
y_val_raw      = y_raw[split:]

# ═══════════════════════════════════════════════════════════════════════
# 2. Iterate over parameter sets
# ═══════════════════════════════════════════════════════════════════════
results = []

for idx, cfg in enumerate(PARAM_SETS, 1):
    tag = f"Run-{idx:02d}  (win={cfg['window']}, filt={cfg['filters']}, pool={cfg['pool']})"
    print(f"\n{tag}\n" + "─" * len(tag))

    # --- prepare windows for this window size
    X_tr, y_tr = make_windows(X_train_scaled, y_train_raw, cfg["window"])
    X_va, y_va = make_windows(X_val_scaled,   y_val_raw,   cfg["window"])

    # --- build & train
    tf.keras.backend.clear_session()
    model = build_model(cfg, n_features)
    model.compile(optimizer=keras.optimizers.Adam(cfg["lr"]),
                  loss="binary_crossentropy")

    cb = [keras.callbacks.EarlyStopping(patience=EARLY_STOP,
                                        restore_best_weights=True,
                                        verbose=0)]
    model.fit(X_tr, y_tr,
              epochs=EPOCHS,
              batch_size=cfg["batch"],
              validation_data=(X_va, y_va),
              callbacks=cb,
              verbose=0)

    # --- evaluate
    prob = model.predict(X_va, verbose=0).ravel()
    pred = (prob >= 0.5).astype(int)

    prec = precision_score(y_va, pred, zero_division=0)
    rec  = recall_score(y_va, pred,    zero_division=0)
    f1   = f1_score(y_va, pred,        zero_division=0)

    print(f"Precision : {prec:5.3f}   Recall : {rec:5.3f}   F1 : {f1:5.3f}")

    results.append({
        **cfg,
        "precision": prec,
        "recall"   : rec,
        "f1"       : f1,
        "auc"      : roc_auc_score(y_va, prob)
    })

# ═══════════════════════════════════════════════════════════════════════
# 3. Leaderboard
# ═══════════════════════════════════════════════════════════════════════
print("\n🏆  Leaderboard (sorted by F1)")
results_sorted = sorted(results, key=lambda d: d["f1"], reverse=True)

for rk, res in enumerate(results_sorted, 1):
    print(f"{rk:>2}. F1={res['f1']:.3f}  "
          f"P={res['precision']:.3f}  R={res['recall']:.3f}  "
          f"(win={res['window']}, filt={res['filters']}, pool={res['pool']})")

# ═══════════════════════════════════════════════════════════════════════
# 4. Optional – save summary JSON
# ═══════════════════════════════════════════════════════════════════════
summary_path = "cnn_param_comparison_summary.json"
with open(summary_path, "w") as fp:
    json.dump({
        "timestamp" : datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "metrics"   : results_sorted
    }, fp, indent=2)

print(f"\n📑 Comparison summary saved → {summary_path}")


📊 Loading & scaling data …

Run-01  (win=12, filt=64, pool=gap)
───────────────────────────────────
Precision : 0.531   Recall : 0.494   F1 : 0.512

Run-02  (win=30, filt=96, pool=gmp)
───────────────────────────────────
Precision : 0.510   Recall : 0.105   F1 : 0.174

Run-03  (win=36, filt=64, pool=gmp)
───────────────────────────────────
Precision : 0.523   Recall : 0.576   F1 : 0.548

Run-04  (win=24, filt=32, pool=gmp)
───────────────────────────────────
Precision : 0.518   Recall : 0.832   F1 : 0.638

Run-05  (win=30, filt=64, pool=gmp)
───────────────────────────────────
Precision : 0.615   Recall : 0.010   F1 : 0.019

Run-06  (win=24, filt=32, pool=gmp)
───────────────────────────────────
Precision : 0.522   Recall : 0.661   F1 : 0.583

Run-07  (win=18, filt=96, pool=gmp)
───────────────────────────────────
Precision : 0.515   Recall : 0.565   F1 : 0.539

Run-08  (win=36, filt=64, pool=gmp)
───────────────────────────────────
Precision : 0.532   Recall : 0.266   F1 : 0.354

Run-

In [ ]:
 {'window': 24, 'batch': 64, 'lr': 0.00056, 'filters': 32, 'kernel': 4,
     'act': 'elu', 'dropout': 0.180, 'dense': 128, 'l2': 1e-5,
     'pool': 'gap', 'extra_dense': True, 'conv_blocks': 2, 'epochs': 20}

In [9]:
"""
cnn_final_train_predict.py
──────────────────────────
Trains a 1-D CNN with optimal hyperparameters and generates predictions CSV.
Uses the best parameters found during optimization for final model training.
"""

import os, json, joblib, warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             roc_auc_score, confusion_matrix, classification_report)

# ═══════════════ Setup ══════════════════════════════════════════════
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ═══════════════ Configuration ══════════════════════════════════════
CSV_PATH = Path(r"C:/Users/ADMIN/Desktop/Coding_projects/stock_market_prediction/"
                r"Stock-Market-Prediction/data/processed/"
                r"gemini_btc_with_features_4h.csv")

# Output files
MODEL_OUT = "cnn_optimal_final.h5"
SCALER_OUT = "cnn_scaler_final.pkl"
PREDICTIONS_OUT = "cnn_predictions.csv"
SUMMARY_JSON = "cnn_final_training_summary.json"

# Optimal hyperparameters
OPTIMAL_PARAMS = {
    'window': 24,
    'batch': 64, 
    'lr': 0.00056,
    'filters': 32,
    'kernel': 4,
    'act': 'elu',
    'dropout': 0.180,
    'dense': 128,
    'l2': 1e-5,
    'pool': 'gap',
    'extra_dense': True,
    'conv_blocks': 2,
    'epochs': 50  # Increased from 20 for better optimization
}

VAL_FRAC = 0.20
EARLY_STOP = 15  # Increased patience for longer training
ALPHA = 2.0  # For weighted F1

# Complete DROP_COLS list
DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower', 'resistance_level',
    'support_level', 'high_low', 'high_close', 'low_close', 'true_range', 'volume_mean_20',
    'MACD_line', 'MACD_signal', 'bollinger_width', 'volatility_regime', 'CCI', 'stoch_%D',
    'parkinson_vol', 'ema_cross_down', 'macd_cross_down', 'vol_spike_1_5x', 'near_upper_band',
    'near_lower_band', 'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive', 'volume_breakout',
    'volume_breakdown', 'stoch_overbought', 'stoch_oversold', 'cci_overbought', 'cci_oversold',
    'trending_market', 'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6', 'bearish_scenario_1',
    'bearish_scenario_2', 'bearish_scenario_3', 'bearish_scenario_4', 'bearish_scenario_6',
    'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'close'
]

# ═══════════════ Helper Functions ═══════════════════════════════════
def weighted_f1(y_true, y_pred, alpha=ALPHA):
    """Calculate weighted F1 score"""
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    return 0.0 if p + r == 0 else (1 + alpha) * p * r / (alpha * p + r)

def make_windows(arr, labels, win):
    """Create sliding windows for time series data"""
    xs, ys = [], []
    for i in range(win, len(arr)):
        xs.append(arr[i-win:i])
        ys.append(labels[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

def build_model(win, n_features):
    """Build CNN model with optimal parameters"""
    params = OPTIMAL_PARAMS
    
    inp = layers.Input(shape=(win, n_features))
    x = inp

    # Multi-block CNN architecture
    for b in range(params['conv_blocks']):
        f = params['filters'] * (2 ** b)
        y = layers.Conv1D(f, params['kernel'], padding="causal",
                          activation=params['act'],
                          kernel_regularizer=regularizers.l2(params['l2']))(x)
        y = layers.BatchNormalization()(y)
        
        # Residual connections for multi-block
        if params['conv_blocks'] > 1:
            if x.shape[-1] != y.shape[-1]:
                x = layers.Conv1D(f, 1, padding="same")(x)
            x = layers.add([x, y])
        else:
            x = y

    # Global pooling
    if params['pool'] == 'gmp':
        x = layers.GlobalMaxPooling1D()(x)
    else:
        x = layers.GlobalAveragePooling1D()(x)
    
    x = layers.Dropout(params['dropout'])(x)

    # Extra dense layer
    if params['extra_dense']:
        x = layers.Dense(params['dense'], activation=params['act'],
                         kernel_regularizer=regularizers.l2(params['l2']))(x)
        x = layers.Dropout(params['dropout'])(x)

    out = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inp, out)

# ═══════════════ 1. Data Loading ═══════════════════════════════════
print("📊 Loading & preprocessing data...")

df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()

X_raw = df.drop(columns="target").values
y_raw = df["target"].astype(int).values
n_features = X_raw.shape[1]

# Time-based train/validation split
split_idx = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split_idx])

X_train_s = scaler.transform(X_raw[:split_idx]).astype(np.float32)
X_val_s = scaler.transform(X_raw[split_idx:]).astype(np.float32)
y_train = y_raw[:split_idx]
y_val = y_raw[split_idx:]

# Create windows
win = OPTIMAL_PARAMS['window']
X_train, y_train = make_windows(X_train_s, y_train, win)
X_val, y_val = make_windows(X_val_s, y_val, win)

print(f"   Total samples: {len(df):,}")
print(f"   Train windows: {len(X_train):,}")
print(f"   Val windows: {len(X_val):,}")
print(f"   Features: {n_features}")
print(f"   Window size: {win}")

# ═══════════════ 2. Model Building ══════════════════════════════════
print("\n🏗️ Building model with optimal parameters...")
print(f"   Architecture: {OPTIMAL_PARAMS['conv_blocks']} conv blocks, "
      f"{OPTIMAL_PARAMS['filters']} filters, kernel={OPTIMAL_PARAMS['kernel']}")
print(f"   Activation: {OPTIMAL_PARAMS['act']}, Pool: {OPTIMAL_PARAMS['pool']}")
print(f"   Dropout: {OPTIMAL_PARAMS['dropout']}, L2: {OPTIMAL_PARAMS['l2']}")

tf.keras.backend.clear_session()
model = build_model(win, n_features)
model.compile(optimizer=keras.optimizers.Adam(OPTIMAL_PARAMS['lr']),
              loss="binary_crossentropy",
              metrics=["accuracy"])

# ═══════════════ 3. Training ════════════════════════════════════════
callbacks = [
    keras.callbacks.EarlyStopping(patience=EARLY_STOP,
                                  restore_best_weights=True,
                                  verbose=1),
    keras.callbacks.ReduceLROnPlateau(factor=0.5,
                                      patience=5,
                                      verbose=1,
                                      min_lr=1e-7)
]

print("\n🚀 Training model...")
history = model.fit(X_train, y_train,
                    epochs=OPTIMAL_PARAMS['epochs'],
                    batch_size=OPTIMAL_PARAMS['batch'],
                    validation_data=(X_val, y_val),
                    callbacks=callbacks,
                    verbose=2)

# ═══════════════ 4. Evaluation ══════════════════════════════════════
print("\n📊 Evaluating model...")
prob_up = model.predict(X_val, verbose=0).ravel()
prob_down = 1 - prob_up
pred = (prob_up >= 0.5).astype(int)

# Metrics
precision = precision_score(y_val, pred, zero_division=0)
recall = recall_score(y_val, pred, zero_division=0)
wf1 = weighted_f1(y_val, pred)
accuracy = accuracy_score(y_val, pred)
auc = roc_auc_score(y_val, prob_up)

metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "weighted_f1_a2": wf1,
    "auc": auc
}

print("\n── Validation Metrics ──")
for k, v in metrics.items():
    print(f"{k:20s}: {v:6.3f}")

print("\nClassification Report:")
print(classification_report(y_val, pred, target_names=["Down", "Up"]))

cm = confusion_matrix(y_val, pred)
print("\nConfusion Matrix:")
print(cm)

# ═══════════════ 5. Generate Predictions CSV ═══════════════════════
print("\n📁 Generating predictions CSV...")

# Get validation timestamps (accounting for window offset)
val_timestamps = df.index[split_idx + win:]

# Create predictions dataframe
predictions_df = pd.DataFrame({
    'timestamp': val_timestamps,
    'prob_up': prob_up,
    'prob_down': prob_down,
    'winning_prob': np.maximum(prob_up, prob_down),
    'prediction': pred,
    'actual': y_val
})

# Format the CSV exactly as requested
predictions_df['timestamp'] = predictions_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Save to CSV with proper column formatting
predictions_df.to_csv(PREDICTIONS_OUT, index=False, float_format='%.6f')

print(f"✅ Predictions saved → {PREDICTIONS_OUT}")
print(f"   Total predictions: {len(predictions_df):,}")

# Show sample predictions
print("\n📋 Sample predictions:")
print(predictions_df.head(10).to_string(index=False))

# ═══════════════ 6. Save Model & Summary ═══════════════════════════
print("\n💾 Saving artifacts...")

# Save model and scaler
keras.models.save_model(model, MODEL_OUT)
joblib.dump(scaler, SCALER_OUT)

# Save comprehensive summary
summary = {
    "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "optimal_parameters": OPTIMAL_PARAMS,
    "dataset_info": {
        "total_samples": len(df),
        "train_samples": len(X_train),
        "val_samples": len(X_val),
        "features": n_features,
        "window_size": win,
        "train_period": f"{df.index[0]} to {df.index[split_idx-1]}",
        "val_period": f"{df.index[split_idx]} to {df.index[-1]}"
    },
    "metrics": {k: float(v) for k, v in metrics.items()},
    "confusion_matrix": cm.tolist(),
    "class_distribution": {
        "train_positive_rate": float(np.mean(y_train)),
        "val_positive_rate": float(np.mean(y_val)),
        "train_counts": [int(np.sum(y_train == 0)), int(np.sum(y_train == 1))],
        "val_counts": [int(np.sum(y_val == 0)), int(np.sum(y_val == 1))]
    },
    "training_info": {
        "epochs_trained": len(history.history['loss']),
        "final_train_loss": float(history.history['loss'][-1]),
        "final_val_loss": float(history.history['val_loss'][-1]),
        "best_val_loss": float(min(history.history['val_loss']))
    }
}

with open(SUMMARY_JSON, "w") as fp:
    json.dump(summary, fp, indent=2)

print(f"✅ Model saved   → {MODEL_OUT}")
print(f"✅ Scaler saved  → {SCALER_OUT}")
print(f"✅ Summary saved → {SUMMARY_JSON}")

# ═══════════════ 7. Final Report ═══════════════════════════════════
print(f"\n🎉 Training Complete!")
print(f"═" * 50)
print(f"📈 Final Performance:")
print(f"   Weighted F1 (α={ALPHA}): {wf1:.3f}")
print(f"   Accuracy: {accuracy:.3f}")
print(f"   Precision: {precision:.3f}")
print(f"   Recall: {recall:.3f}")
print(f"   AUC: {auc:.3f}")
print(f"")
print(f"📁 Files Generated:")
print(f"   • {MODEL_OUT} - Trained model")
print(f"   • {SCALER_OUT} - Feature scaler")
print(f"   • {PREDICTIONS_OUT} - Validation predictions ({len(predictions_df):,} rows)")
print(f"   • {SUMMARY_JSON} - Training summary")
print(f"")
print(f"✨ Ready for production use!")

📊 Loading & preprocessing data...
   Total samples: 15,855
   Train windows: 12,660
   Val windows: 3,147
   Features: 19
   Window size: 24

🏗️ Building model with optimal parameters...
   Architecture: 2 conv blocks, 32 filters, kernel=4
   Activation: elu, Pool: gap
   Dropout: 0.18, L2: 1e-05

🚀 Training model...
Epoch 1/50
198/198 - 2s - 10ms/step - accuracy: 0.5000 - loss: 0.7179 - val_accuracy: 0.4944 - val_loss: 0.6981 - learning_rate: 5.6000e-04
Epoch 2/50
198/198 - 1s - 3ms/step - accuracy: 0.5066 - loss: 0.7033 - val_accuracy: 0.5122 - val_loss: 0.6963 - learning_rate: 5.6000e-04
Epoch 3/50
198/198 - 1s - 3ms/step - accuracy: 0.5087 - loss: 0.7006 - val_accuracy: 0.5195 - val_loss: 0.6963 - learning_rate: 5.6000e-04
Epoch 4/50
198/198 - 1s - 3ms/step - accuracy: 0.5059 - loss: 0.6996 - val_accuracy: 0.5151 - val_loss: 0.6969 - learning_rate: 5.6000e-04
Epoch 5/50
198/198 - 1s - 3ms/step - accuracy: 0.5083 - loss: 0.6982 - val_accuracy: 0.5202 - val_loss: 0.6962 - learning_ra


── Validation Metrics ──
accuracy            :  0.514
precision           :  0.525
recall              :  0.713
weighted_f1_a2      :  0.637
auc                 :  0.506

Classification Report:
              precision    recall  f1-score   support

        Down       0.48      0.30      0.37      1503
          Up       0.53      0.71      0.60      1644

    accuracy                           0.51      3147
   macro avg       0.51      0.50      0.49      3147
weighted avg       0.51      0.51      0.49      3147


Confusion Matrix:
[[ 444 1059]
 [ 472 1172]]

📁 Generating predictions CSV...
✅ Predictions saved → cnn_predictions.csv
   Total predictions: 3,147

📋 Sample predictions:
          timestamp  prob_up  prob_down  winning_prob  prediction  actual
2023-10-20 16:00:00 0.520832   0.479168      0.520832           1       1
2023-10-20 20:00:00 0.520022   0.479978      0.520022           1       1
2023-10-21 00:00:00 0.527823   0.472177      0.527823           1       0
2023-10-21

In [10]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\Final_runs_csv\cnn_predictions.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")


📊 Evaluation at threshold 0.5:
Precision: 0.525
Recall   : 0.713
F1 Score : 0.605
